In [2]:
# 自动计算cell的计算时间
%load_ext autotime

%matplotlib inline
%config InlineBackend.figure_format='svg' #矢量图设置，让绘图更清晰

time: 735 ms (started: 2021-08-26 19:21:23 +08:00)


In [3]:
%%bash

# 增加更新
git add *.ipynb *.md

git remote -v

git commit -m '更新 #8  Aug 26, 2021'

#git push origin master
git push

origin	git@github.com:ustchope/keras_developer_guides-.git (fetch)
origin	git@github.com:ustchope/keras_developer_guides-.git (push)
[main 98ea9e4] 更新 #8  Aug 26, 2021
 5 files changed, 3067 insertions(+), 38 deletions(-)
 create mode 100644 "\344\275\277\347\224\250 TensorFlow Cloud \350\256\255\347\273\203 Keras \346\250\241\345\236\213.ipynb"
 create mode 100644 "\345\244\232 GPU \345\222\214\345\210\206\345\270\203\345\274\217\350\256\255\347\273\203.ipynb"
 create mode 100644 "\350\277\201\347\247\273\345\255\246\344\271\240\345\222\214\345\276\256\350\260\203.ipynb"
 create mode 100644 "\351\235\242\345\220\221\347\240\224\347\251\266\344\272\272\345\221\230\347\232\204 Keras \347\256\200\344\273\213.ipynb"


To git@github.com:ustchope/keras_developer_guides-.git
   93164f7..98ea9e4  main -> main


time: 6.14 s (started: 2021-08-26 19:22:03 +08:00)


In [4]:
#设置使用的gpu
import tensorflow as tf
from tensorflow import keras

gpus = tf.config.list_physical_devices("GPU")

if gpus:
   
    gpu0 = gpus[0] #如果有多个GPU，仅使用第0个GPU
    tf.config.experimental.set_memory_growth(gpu0, True) #设置GPU显存用量按需使用
    # 或者也可以设置GPU显存为固定使用量(例如：4G)
    #tf.config.experimental.set_virtual_device_configuration(gpu0,
    #    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)]) 
    tf.config.set_visible_devices([gpu0],"GPU")

time: 5.41 s (started: 2021-08-26 19:22:09 +08:00)


# 设置

In [5]:
import tensorflow as tf
from tensorflow import keras

time: 498 µs (started: 2021-08-26 19:22:33 +08:00)


# 介绍
你是机器学习研究员吗？ 您是否在 NeurIPS 上发表文章并推动 CV 和 NLP 的最新技术？ 本指南将作为您对核心 Keras 和 TensorFlow API 概念的首次介绍。

在本指南中，您将了解：
* TensorFlow 中的张量、变量和梯度
* 通过子类化 Layer 类来创建层
* 编写低级训练循环
* 通过 add_loss() 方法跟踪由层创建的损失
* 在低级训练循环中跟踪指标
* 使用编译的 tf.function 加速执行
* 在训练或推理模式下执行层
* Keras 函数式 API

您还将在两个端到端研究示例中看到 Keras API 的实际应用：变分自动编码器和超网络。

# 张量
TensorFlow 是用于可微分编程的基础设施层。 从本质上讲，它是一个用于操作 N 维数组（张量）的框架，很像 NumPy。

但是，NumPy 和 TensorFlow 之间存在三个主要区别：
* TensorFlow 可以利用硬件加速器，例如 GPU 和 TPU。
* TensorFlow 可以自动计算任意可微张量表达式的梯度。
* TensorFlow 计算可以分布在一台机器上的大量设备上，也可以分布在大量机器上（每个机器可能有多个设备）。

让我们来看看 TensorFlow 的核心对象：Tensor。

这是一个常数张量：

In [6]:
x = tf.constant([[5, 2], [1, 3]])
print(x)

tf.Tensor(
[[5 2]
 [1 3]], shape=(2, 2), dtype=int32)
time: 1.71 s (started: 2021-08-26 19:23:59 +08:00)


您可以通过调用 .numpy() 以 NumPy 数组的形式获取其值：

In [7]:
x.numpy()

array([[5, 2],
       [1, 3]], dtype=int32)

time: 14.3 ms (started: 2021-08-26 19:24:20 +08:00)


很像 NumPy 数组，它具有属性 dtype 和 shape：

In [8]:
print("dtype:", x.dtype)
print("shape:", x.shape)

dtype: <dtype: 'int32'>
shape: (2, 2)
time: 734 µs (started: 2021-08-26 19:24:39 +08:00)


创建常量张量的常用方法是通过 tf.ones 和 tf.zeros（就像 np.ones 和 np.zeros）：

In [9]:
print(tf.ones(shape=(2, 1)))
print(tf.zeros(shape=(2, 1)))

tf.Tensor(
[[1.]
 [1.]], shape=(2, 1), dtype=float32)
tf.Tensor(
[[0.]
 [0.]], shape=(2, 1), dtype=float32)
time: 17 ms (started: 2021-08-26 19:25:01 +08:00)


您还可以创建随机常数张量：

In [11]:
x = tf.random.normal(shape=(2, 2), mean=0.0, stddev=1.0)

x = tf.random.uniform(shape=(2, 2), minval=0, maxval=10, dtype="int32")

time: 19.3 ms (started: 2021-08-26 19:25:26 +08:00)


# 变量
变量是用于存储可变状态（例如神经网络的权重）的特殊张量。 您使用一些初始值创建一个变量：

In [12]:
initial_value = tf.random.normal(shape=(2, 2))
a = tf.Variable(initial_value)
print(a)

<tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[-1.0114193 , -0.8392619 ],
       [-0.14134249,  0.508994  ]], dtype=float32)>
time: 10.2 ms (started: 2021-08-26 19:25:55 +08:00)


您可以使用 `.assign(value)`、`.assign_add(increment)` 或 `.assign_sub(decrement)` 方法更新变量的值：

In [13]:
new_value = tf.random.normal(shape=(2, 2))
a.assign(new_value)
for i in range(2):
    for j in range(2):
        assert a[i, j] == new_value[i, j]

added_value = tf.random.normal(shape=(2, 2))
a.assign_add(added_value)
for i in range(2):
    for j in range(2):
        assert a[i, j] == new_value[i, j] + added_value[i, j]

time: 40.4 ms (started: 2021-08-26 19:26:40 +08:00)


# 在 TensorFlow 中进行数学运算
如果您使用过 NumPy，那么在 TensorFlow 中进行数学运算看起来会非常熟悉。 主要区别在于您的 TensorFlow 代码可以在 GPU 和 TPU 上运行。

In [14]:
a = tf.random.normal(shape=(2, 2))
b = tf.random.normal(shape=(2, 2))

c = a + b
d = tf.square(c)
e = tf.exp(d)

time: 12.1 ms (started: 2021-08-26 19:27:04 +08:00)


# 梯度
这是与 NumPy 的另一个重大区别：您可以自动检索任何可微表达式的梯度。

只需打开一个 `GradientTape`，通过`tape.watch()` 开始“观察”一个张量，并使用这个张量作为输入组成一个可微的表达式：

In [15]:
a = tf.random.normal(shape=(2, 2))
b = tf.random.normal(shape=(2, 2))

with tf.GradientTape() as tape:
    tape.watch(a)  # Start recording the history of operations applied to `a`
    c = tf.sqrt(tf.square(a) + tf.square(b))  # Do some math using `a`
    # What's the gradient of `c` with respect to `a`?
    dc_da = tape.gradient(c, a)
    print(dc_da)

tf.Tensor(
[[ 0.15414332  0.41588616]
 [ 0.98600286 -0.69903207]], shape=(2, 2), dtype=float32)
time: 19.9 ms (started: 2021-08-26 19:27:42 +08:00)


默认情况下，变量是自动监视的，因此您无需手动监视它们：

In [16]:
a = tf.Variable(a)

with tf.GradientTape() as tape:
    c = tf.sqrt(tf.square(a) + tf.square(b))
    dc_da = tape.gradient(c, a)
    print(dc_da)

tf.Tensor(
[[ 0.15414332  0.41588616]
 [ 0.98600286 -0.69903207]], shape=(2, 2), dtype=float32)
time: 14.2 ms (started: 2021-08-26 19:28:05 +08:00)


请注意，您可以通过嵌套磁带来计算高阶导数：

In [17]:
with tf.GradientTape() as outer_tape:
    with tf.GradientTape() as tape:
        c = tf.sqrt(tf.square(a) + tf.square(b))
        dc_da = tape.gradient(c, a)
    d2c_da2 = outer_tape.gradient(dc_da, a)
    print(d2c_da2)

tf.Tensor(
[[0.5860508  1.0559046 ]
 [0.01313475 0.30708778]], shape=(2, 2), dtype=float32)
time: 23 ms (started: 2021-08-26 19:28:26 +08:00)


# Keras 层
TensorFlow 是用于可微编程的基础设施层，处理张量、变量和梯度，而 Keras 是用于深度学习的用户界面，处理层、模型、优化器、损失函数、指标等。

Keras 作为 TensorFlow 的高级 API：Keras 使 TensorFlow 变得简单而高效。

Layer 类是 Keras 中的基本抽象。 一个层封装了一个状态（权重）和一些计算（在调用方法中定义）。

一个简单的层看起来像这样：

In [19]:
class Linear(keras.layers.Layer):
    """y = w.x + b"""

    def __init__(self, units=32, input_dim=32):
        super(Linear, self).__init__()
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(
            initial_value=w_init(shape=(input_dim, units), dtype="float32"),
            trainable=True,
        )
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(
            initial_value=b_init(shape=(units,), dtype="float32"), trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

time: 888 µs (started: 2021-08-26 19:29:27 +08:00)


您可以像使用 Python 函数一样使用 Layer 实例：

In [20]:
# Instantiate our layer.
linear_layer = Linear(units=4, input_dim=2)

# The layer can be treated as a function.
# Here we call it on some data.
y = linear_layer(tf.ones((2, 2)))
assert y.shape == (2, 4)

time: 2.32 s (started: 2021-08-26 19:30:15 +08:00)


权重变量（在 `__init__` 中创建）在 `weights` 属性下自动跟踪：

In [21]:
assert linear_layer.weights == [linear_layer.w, linear_layer.b]

time: 399 µs (started: 2021-08-26 19:30:55 +08:00)


您有许多可用的内置层，从 Dense 到 Conv2D，再到 LSTM，再到更高级的层，如 Conv3DTranspose 或 ConvLSTM2D。 明智地重用内置功能。

# 层权重创建
`self.add_weight()` 方法为您提供了创建权重的快捷方式：

In [22]:
class Linear(keras.layers.Layer):
    """y = w.x + b"""

    def __init__(self, units=32):
        super(Linear, self).__init__()
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
        )
        self.b = self.add_weight(
            shape=(self.units,), initializer="random_normal", trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b


# Instantiate our lazy layer.
linear_layer = Linear(4)

# This will also call `build(input_shape)` and create the weights.
y = linear_layer(tf.ones((2, 2)))

time: 17.7 ms (started: 2021-08-26 19:32:06 +08:00)


# 层梯度
您可以通过在 `GradientTape` 中调用图层来自动检索图层权重的梯度。 使用这些梯度，您可以手动或使用优化器对象更新层的权重。 当然，如果需要，您可以在使用之前修改梯度。

In [23]:
# Prepare a dataset.
(x_train, y_train), _ = tf.keras.datasets.mnist.load_data()
dataset = tf.data.Dataset.from_tensor_slices(
    (x_train.reshape(60000, 784).astype("float32") / 255, y_train)
)
dataset = dataset.shuffle(buffer_size=1024).batch(64)

# Instantiate our linear layer (defined above) with 10 units.
linear_layer = Linear(10)

# Instantiate a logistic loss function that expects integer targets.
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Instantiate an optimizer.
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)

# Iterate over the batches of the dataset.
for step, (x, y) in enumerate(dataset):

    # Open a GradientTape.
    with tf.GradientTape() as tape:

        # Forward pass.
        logits = linear_layer(x)

        # Loss value for this batch.
        loss = loss_fn(y, logits)

    # Get gradients of the loss wrt the weights.
    gradients = tape.gradient(loss, linear_layer.trainable_weights)

    # Update the weights of our linear layer.
    optimizer.apply_gradients(zip(gradients, linear_layer.trainable_weights))

    # Logging.
    if step % 100 == 0:
        print("Step:", step, "Loss:", float(loss))

Step: 0 Loss: 2.419081926345825
Step: 100 Loss: 2.3437397480010986
Step: 200 Loss: 2.1802117824554443
Step: 300 Loss: 2.0649428367614746
Step: 400 Loss: 2.0387189388275146
Step: 500 Loss: 1.9206409454345703
Step: 600 Loss: 1.8478238582611084
Step: 700 Loss: 1.6928911209106445
Step: 800 Loss: 1.7000257968902588
Step: 900 Loss: 1.6581981182098389
time: 8.31 s (started: 2021-08-26 19:34:27 +08:00)


# 可训练和不可训练的权重
由层创建的权重可以是可训练的，也可以是不可训练的。 它们分别暴露在 `trainable_weights` 和 `non_trainable_weights` 中。 这是一个具有不可训练权重的层：

In [24]:
class ComputeSum(keras.layers.Layer):
    """Returns the sum of the inputs."""

    def __init__(self, input_dim):
        super(ComputeSum, self).__init__()
        # Create a non-trainable weight.
        self.total = tf.Variable(initial_value=tf.zeros((input_dim,)), trainable=False)

    def call(self, inputs):
        self.total.assign_add(tf.reduce_sum(inputs, axis=0))
        return self.total


my_sum = ComputeSum(2)
x = tf.ones((2, 2))

y = my_sum(x)
print(y.numpy())  # [2. 2.]

y = my_sum(x)
print(y.numpy())  # [4. 4.]

assert my_sum.weights == [my_sum.total]
assert my_sum.non_trainable_weights == [my_sum.total]
assert my_sum.trainable_weights == []

[2. 2.]
[4. 4.]
time: 15.8 ms (started: 2021-08-26 19:36:30 +08:00)


# 拥有层的层
层可以递归嵌套以创建更大的计算块。 每个层将跟踪其子层（可训练和不可训练）的权重。

In [25]:
# 让我们重用 Linear 类
# 使用我们上面定义的 `build` 方法。


class MLP(keras.layers.Layer):
    """Simple stack of Linear layers."""

    def __init__(self):
        super(MLP, self).__init__()
        self.linear_1 = Linear(32)
        self.linear_2 = Linear(32)
        self.linear_3 = Linear(10)

    def call(self, inputs):
        x = self.linear_1(inputs)
        x = tf.nn.relu(x)
        x = self.linear_2(x)
        x = tf.nn.relu(x)
        return self.linear_3(x)


mlp = MLP()

# The first call to the `mlp` object will create the weights.
y = mlp(tf.ones(shape=(3, 64)))

# Weights are recursively tracked.
assert len(mlp.weights) == 6

time: 23.7 ms (started: 2021-08-26 19:38:09 +08:00)


请注意，我们上面手动创建的 MLP 等效于以下内置选项：

In [26]:
mlp = keras.Sequential(
    [
        keras.layers.Dense(32, activation=tf.nn.relu),
        keras.layers.Dense(32, activation=tf.nn.relu),
        keras.layers.Dense(10),
    ]
)

time: 15 ms (started: 2021-08-26 19:38:37 +08:00)


# 跟踪层造成的损失
层可以通过`add_loss()` 方法在前向传递期间产生损失。 这对于正则化损失特别有用。 子层创建的损失由父层递归跟踪。

这是一个创建活动正则化损失的层：

In [27]:
class ActivityRegularization(keras.layers.Layer):
    """Layer that creates an activity sparsity regularization loss."""

    def __init__(self, rate=1e-2):
        super(ActivityRegularization, self).__init__()
        self.rate = rate

    def call(self, inputs):
        # We use `add_loss` to create a regularization loss
        # that depends on the inputs.
        self.add_loss(self.rate * tf.reduce_sum(inputs))
        return inputs

time: 1.1 ms (started: 2021-08-26 19:41:55 +08:00)


任何包含该层的模型都将跟踪此正则化损失：

In [28]:
# Let's use the loss layer in a MLP block.


class SparseMLP(keras.layers.Layer):
    """Stack of Linear layers with a sparsity regularization loss."""

    def __init__(self):
        super(SparseMLP, self).__init__()
        self.linear_1 = Linear(32)
        self.regularization = ActivityRegularization(1e-2)
        self.linear_3 = Linear(10)

    def call(self, inputs):
        x = self.linear_1(inputs)
        x = tf.nn.relu(x)
        x = self.regularization(x)
        return self.linear_3(x)


mlp = SparseMLP()
y = mlp(tf.ones((10, 10)))

print(mlp.losses)  # List containing one float32 scalar

[<tf.Tensor: shape=(), dtype=float32, numpy=0.20129697>]
time: 21.4 ms (started: 2021-08-26 19:43:58 +08:00)


这些损失在每次前向传递开始时由顶层清除——它们不会累积。 layer.losses 总是只包含在最后一次向前传递期间产生的损失。 在编写训练循环时，您通常会在计算梯度之前通过对它们求和来使用这些损失。

In [29]:
# Losses correspond to the *last* forward pass.
mlp = SparseMLP()
mlp(tf.ones((10, 10)))
assert len(mlp.losses) == 1
mlp(tf.ones((10, 10)))
assert len(mlp.losses) == 1  # No accumulation.

# Let's demonstrate how to use these losses in a training loop.

# Prepare a dataset.
(x_train, y_train), _ = tf.keras.datasets.mnist.load_data()
dataset = tf.data.Dataset.from_tensor_slices(
    (x_train.reshape(60000, 784).astype("float32") / 255, y_train)
)
dataset = dataset.shuffle(buffer_size=1024).batch(64)

# A new MLP.
mlp = SparseMLP()

# Loss and optimizer.
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)

for step, (x, y) in enumerate(dataset):
    with tf.GradientTape() as tape:

        # Forward pass.
        logits = mlp(x)

        # 该批次的外部损失值。
        loss = loss_fn(y, logits)

        # 添加前向传递期间产生的损失。
        loss += sum(mlp.losses)

        # 获取权重的损失梯度。
        gradients = tape.gradient(loss, mlp.trainable_weights)

    # 更新我们的线性层的权重。
    optimizer.apply_gradients(zip(gradients, mlp.trainable_weights))

    # Logging.
    if step % 100 == 0:
        print("Step:", step, "Loss:", float(loss))

Step: 0 Loss: 5.366790771484375
Step: 100 Loss: 2.538097381591797
Step: 200 Loss: 2.395237922668457
Step: 300 Loss: 2.3674986362457275
Step: 400 Loss: 2.3502919673919678
Step: 500 Loss: 2.3570244312286377
Step: 600 Loss: 2.326946258544922
Step: 700 Loss: 2.3309104442596436
Step: 800 Loss: 2.314706802368164
Step: 900 Loss: 2.3405709266662598
time: 9.93 s (started: 2021-08-26 19:46:50 +08:00)


# 跟踪衡量指标
Keras 提供了广泛的内置指标，例如 `tf.keras.metrics.AUC` 或 `tf.keras.metrics.PrecisionAtRecall`。 用几行代码创建自己的指标也很容易。

要在自定义训练循环中使用指标，您需要：
* 实例化度量对象，例如 `metric = tf.keras.metrics.AUC()`
* 为每批数据调用其 `metric.udpate_state(targets, predictions)` 方法
* 通过 `metric.result()` 查询其结果
* 通过 `metric.reset_states()` 在 epoch 结束或评估开始时重置指标的状态

这是一个简单的例子：

In [31]:
# 实例化一个度量对象
accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

# 准备我们的层、损失和优化器。
model = keras.Sequential(
    [
        keras.layers.Dense(32, activation="relu"),
        keras.layers.Dense(32, activation="relu"),
        keras.layers.Dense(10),
    ]
)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

for epoch in range(2):
    # Iterate over the batches of a dataset.
    for step, (x, y) in enumerate(dataset):
        with tf.GradientTape() as tape:
            logits = model(x)
            # Compute the loss value for this batch.
            loss_value = loss_fn(y, logits)

        # 更新“准确度”指标的状态。
        accuracy.update_state(y, logits)

        # 更新模型的权重以最小化损失值。
        gradients = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(gradients, model.trainable_weights))

        # 记录目前的准确度值。
        if step % 200 == 0:
            print("Epoch:", epoch, "Step:", step)
            print("Total running accuracy so far: %.3f" % accuracy.result())

    # 在Epoch结束时重置指标的状态
    accuracy.reset_states()

Epoch: 0 Step: 0
Total running accuracy so far: 0.031
Epoch: 0 Step: 200
Total running accuracy so far: 0.772
Epoch: 0 Step: 400
Total running accuracy so far: 0.837
Epoch: 0 Step: 600
Total running accuracy so far: 0.863
Epoch: 0 Step: 800
Total running accuracy so far: 0.878
Epoch: 1 Step: 0
Total running accuracy so far: 0.938
Epoch: 1 Step: 200
Total running accuracy so far: 0.940
Epoch: 1 Step: 400
Total running accuracy so far: 0.940
Epoch: 1 Step: 600
Total running accuracy so far: 0.941
Epoch: 1 Step: 800
Total running accuracy so far: 0.942
time: 23.7 s (started: 2021-08-26 19:53:45 +08:00)


除此之外，类似于 `self.add_loss()` 方法，您可以访问层上的 `self.add_metric()` 方法。 它跟踪您传递给它的任何数量的平均值。 您可以通过在任何层或模型上调用 `layer.reset_metrics()` 来重置这些指标的值。

## 编译函数
急切地运行非常适合调试，但通过将计算编译为静态图，您将获得更好的性能。 静态图是研究人员最好的朋友。 您可以通过将其包装在 tf.function 装饰器中来编译任何函数。

In [37]:
# Prepare our layer, loss, and optimizer.
model = keras.Sequential(
    [
        keras.layers.Dense(32, activation="relu"),
        keras.layers.Dense(32, activation="relu"),
        keras.layers.Dense(10),
    ]
)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

# Create a training step function.


@tf.function  # Make it fast.
def train_on_batch(x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = loss_fn(y, logits)
#         gradients = tape.gradient(loss, model.trainable_weights)
#     optimizer.apply_gradients(zip(gradients, model.trainable_weights))
    optimizer.minimize(loss, model.trainable_weights, tape=tape)
    return loss


# Prepare a dataset.
(x_train, y_train), _ = tf.keras.datasets.mnist.load_data()
dataset = tf.data.Dataset.from_tensor_slices(
    (x_train.reshape(60000, 784).astype("float32") / 255, y_train)
)
dataset = dataset.shuffle(buffer_size=1024).batch(64)

for step, (x, y) in enumerate(dataset):
    loss = train_on_batch(x, y)
    if step % 100 == 0:
        print("Step:", step, "Loss:", float(loss))

Step: 0 Loss: 2.329501152038574
Step: 100 Loss: 0.644128680229187
Step: 200 Loss: 0.5418049693107605
Step: 300 Loss: 0.35992997884750366
Step: 400 Loss: 0.34068578481674194
Step: 500 Loss: 0.19066256284713745
Step: 600 Loss: 0.21403372287750244
Step: 700 Loss: 0.2896679937839508
Step: 800 Loss: 0.37784165143966675
Step: 900 Loss: 0.34159013628959656
time: 3.6 s (started: 2021-08-26 20:13:56 +08:00)


# 训练模式&推理模式
某些层，尤其是 BatchNormalization 层和 Dropout 层，在训练和推理过程中具有不同的行为。 对于此类层，标准做法是在 call 方法中公开`training`（布尔值）参数。

通过在调用中公开此参数，您可以启用内置的训练和评估循环（例如 fit）以在训练和推理模式中正确使用该层。

In [33]:
class Dropout(keras.layers.Layer):
    def __init__(self, rate):
        super(Dropout, self).__init__()
        self.rate = rate

    def call(self, inputs, training=None):
        if training:
            return tf.nn.dropout(inputs, rate=self.rate)
        return inputs


class MLPWithDropout(keras.layers.Layer):
    def __init__(self):
        super(MLPWithDropout, self).__init__()
        self.linear_1 = Linear(32)
        self.dropout = Dropout(0.5)
        self.linear_3 = Linear(10)

    def call(self, inputs, training=None):
        x = self.linear_1(inputs)
        x = tf.nn.relu(x)
        x = self.dropout(x, training=training)
        return self.linear_3(x)


mlp = MLPWithDropout()
y_train = mlp(tf.ones((2, 2)), training=True)
y_test = mlp(tf.ones((2, 2)), training=False)

time: 20.9 ms (started: 2021-08-26 19:57:41 +08:00)


# 用于模型构建的函数式 API
要构建深度学习模型，您不必一直使用面向对象的编程。 到目前为止，我们看到的所有层也可以按功能组合，如下所示（我们称之为“函数 API”）：

In [34]:
# 我们使用 `Input` 对象来描述输入的形状和数据类型。
# 这是深度学习相当于*声明一个类型*。
# 形状参数是每个样本； 它不包括批量大小。
# 函数式 API 专注于定义每个样本的转换。
# 我们创建的模型将自动批处理每个样本的转换，以便可以在批量数据上调用它。
inputs = tf.keras.Input(shape=(16,), dtype="float32")

# 我们在这些“类型”对象上调用层
# 并且它们返回更新的类型（新的形状/数据类型）。
x = Linear(32)(inputs)  # 我们正在重用我们之前定义的线性层。
x = Dropout(0.5)(x)  # 我们正在重用我们之前定义的 Dropout 层。
outputs = Linear(10)(x)

# 可以通过指定输入和输出来定义功能性`Model`。
# 模型本身就是一个层，就像任何其他层一样。
model = tf.keras.Model(inputs, outputs)

# 在对任何数据调用之前，功能模型已经具有权重。
# 那是因为我们预先定义了它的输入形状（在`Input`中）。
assert len(model.weights) == 4

# 为了好玩，让我们在一些数据上调用我们的模型。
y = model(tf.ones((2, 16)))
assert y.shape == (2, 10)

# 你可以在 `__call__` 中传递一个 `training` 参数
#（它将被传递到 Dropout 层）。
y = model(tf.ones((2, 16)), training=True)

time: 165 ms (started: 2021-08-26 19:59:53 +08:00)


函数式 API 往往比子类化更简洁，并提供了一些其他优势（与非类型化 OO 开发相比，函数式类型语言提供的优势通常相同）。 但是，它只能用于定义层的 DAG——递归网络应该被定义为层子类。

在此处了解有关 Functional API 的更多信息。

在您的研究工作流程中，您可能经常发现自己混合搭配了面向对象模型和功能模型。

请注意，模型类还具有内置的训练和评估循环（fit() 和evaluate()）。 如果您想将这些循环用于您的 OO 模型，您始终可以对 Model 类进行子类化（它的工作方式与子类化 Layer 完全一样）。

# 端到端实验示例 1：变分自编码器。
以下是您迄今为止学到的一些内容：
* 一个层封装了一个状态（在 `__init__` 或 `build` 中创建）和一些计算（在调用中定义）。
* 层可以递归嵌套以创建新的、更大的计算块。
* 您可以通过打开 `GradientTape`，在磁带范围内调用您的模型，然后检索梯度并通过优化器应用它们来轻松编写高度可破解的训练循环。
* 您可以使用 `@tf.function` 装饰器加速训练循环。
* 层可以通过 `self.add_loss()` 创建和跟踪损失（通常是正则化损失）。

让我们把所有这些东西放在一个端到端的例子中：我们将实现一个变分自动编码器 (VAE)。 我们将在 MNIST 数字上训练它。

我们的 VAE 将是 Layer 的子类，构建为子类 Layer 的嵌套层组合。 它将具有正则化损失（KL 散度）。

下面是我们的模型定义。

首先，我们有一个 Encoder 类，它使用采样层将 MNIST 数字映射到潜在空间三元组 `(z_mean, z_log_var, z)`。

In [43]:
from tensorflow.keras import layers


class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon


class Encoder(layers.Layer):
    """Maps MNIST digits to a triplet (z_mean, z_log_var, z)."""

    def __init__(self, latent_dim=32, intermediate_dim=64, **kwargs):
        super(Encoder, self).__init__(**kwargs)
        self.dense_proj = layers.Dense(intermediate_dim, activation=tf.nn.relu)
        self.dense_mean = layers.Dense(latent_dim)
        self.dense_log_var = layers.Dense(latent_dim)
        self.sampling = Sampling()

    def call(self, inputs):
        x = self.dense_proj(inputs)
        z_mean = self.dense_mean(x)
        z_log_var = self.dense_log_var(x)
        z = self.sampling((z_mean, z_log_var))
        return z_mean, z_log_var, z

time: 2.1 ms (started: 2021-08-26 20:41:26 +08:00)


接下来，我们有一个解码器类，它将概率潜在空间坐标映射回 MNIST 数字。

In [39]:
class Decoder(layers.Layer):
    """Converts z, the encoded digit vector, back into a readable digit."""

    def __init__(self, original_dim, intermediate_dim=64, **kwargs):
        super(Decoder, self).__init__(**kwargs)
        self.dense_proj = layers.Dense(intermediate_dim, activation=tf.nn.relu)
        self.dense_output = layers.Dense(original_dim, activation=tf.nn.sigmoid)

    def call(self, inputs):
        x = self.dense_proj(inputs)
        return self.dense_output(x)

time: 700 µs (started: 2021-08-26 20:16:03 +08:00)


最后，我们的 VariationalAutoEncoder 将编码器和解码器组合在一起，并通过 add_loss() 创建 KL 散度正则化损失。

In [41]:
class VariationalAutoEncoder(layers.Layer):
    """Combines the encoder and decoder into an end-to-end model for training."""

    def __init__(self, original_dim, intermediate_dim=64, latent_dim=32, **kwargs):
        super(VariationalAutoEncoder, self).__init__(**kwargs)
        self.original_dim = original_dim
        self.encoder = Encoder(latent_dim=latent_dim, intermediate_dim=intermediate_dim)
        self.decoder = Decoder(original_dim, intermediate_dim=intermediate_dim)

    def call(self, inputs):
        z_mean, z_log_var, z = self.encoder(inputs)
        reconstructed = self.decoder(z)
        # Add KL divergence regularization loss.
        kl_loss = -0.5 * tf.reduce_mean(
            z_log_var - tf.square(z_mean) - tf.exp(z_log_var) + 1
        )
        self.add_loss(kl_loss)
        return reconstructed

time: 2.18 ms (started: 2021-08-26 20:37:04 +08:00)


现在，让我们编写一个训练循环。 我们的训练步骤用 @tf.function 修饰以编译成一个超快速的图函数。

In [44]:
# Our model.
vae = VariationalAutoEncoder(original_dim=784, intermediate_dim=64, latent_dim=32)

# Loss and optimizer.
loss_fn = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

# Prepare a dataset.
(x_train, _), _ = tf.keras.datasets.mnist.load_data()
dataset = tf.data.Dataset.from_tensor_slices(
    x_train.reshape(60000, 784).astype("float32") / 255
)
dataset = dataset.shuffle(buffer_size=1024).batch(32)


@tf.function
def training_step(x):
    with tf.GradientTape() as tape:
        reconstructed = vae(x)  # Compute input reconstruction.
        # Compute loss.
        loss = loss_fn(x, reconstructed)
        loss += sum(vae.losses)  # Add KLD term.
    # Update the weights of the VAE.
#     grads = tape.gradient(loss, vae.trainable_weights)
#     optimizer.apply_gradients(zip(grads, vae.trainable_weights))
    optimizer.minimize(loss, vae.trainable_weights, tape=tape)
    return loss

losses = []  # Keep track of the losses over time.
for step, x in enumerate(dataset):
    loss = training_step(x)
    # Logging.
    losses.append(float(loss))
    if step % 100 == 0:
        print("Step:", step, "Loss:", sum(losses) / len(losses))

    # Stop after 1000 steps.
    # Training the model to convergence is left
    # as an exercise to the reader.
    if step >= 1000:
        break

Step: 0 Loss: 0.3442484736442566
Step: 100 Loss: 0.1264608879343118
Step: 200 Loss: 0.10004165086580154
Step: 300 Loss: 0.08995694439375519
Step: 400 Loss: 0.0848816904399609
Step: 500 Loss: 0.0816401359132545
Step: 600 Loss: 0.07925445616815531
Step: 700 Loss: 0.07792138239732822
Step: 800 Loss: 0.07670980008707362
Step: 900 Loss: 0.07570658301672052
Step: 1000 Loss: 0.07479102916635952
time: 3.91 s (started: 2021-08-26 20:41:37 +08:00)


如您所见，在 Keras 中构建和训练这种类型的模型是快速且轻松的。

现在，您可能会发现上面的代码有些冗长：我们自己手动处理每一个小细节。 这提供了最大的灵活性，但也需要一些工作。

让我们来看看我们的 VAE 的 Functional API 版本是什么样的：

In [45]:
original_dim = 784
intermediate_dim = 64
latent_dim = 32

# Define encoder model.
original_inputs = tf.keras.Input(shape=(original_dim,), name="encoder_input")
x = layers.Dense(intermediate_dim, activation="relu")(original_inputs)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()((z_mean, z_log_var))
encoder = tf.keras.Model(inputs=original_inputs, outputs=z, name="encoder")

# Define decoder model.
latent_inputs = tf.keras.Input(shape=(latent_dim,), name="z_sampling")
x = layers.Dense(intermediate_dim, activation="relu")(latent_inputs)
outputs = layers.Dense(original_dim, activation="sigmoid")(x)
decoder = tf.keras.Model(inputs=latent_inputs, outputs=outputs, name="decoder")

# Define VAE model.
outputs = decoder(z)
vae = tf.keras.Model(inputs=original_inputs, outputs=outputs, name="vae")

# Add KL divergence regularization loss.
kl_loss = -0.5 * tf.reduce_mean(z_log_var - tf.square(z_mean) - tf.exp(z_log_var) + 1)
vae.add_loss(kl_loss)

time: 116 ms (started: 2021-08-26 20:42:31 +08:00)


简洁多了，对吧？

顺便说一下，Keras 在其模型类（fit() 和evaluate()）上还具有内置的训练和评估循环。 一探究竟：

In [46]:
# Loss and optimizer.
loss_fn = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

# Prepare a dataset.
(x_train, _), _ = tf.keras.datasets.mnist.load_data()
dataset = tf.data.Dataset.from_tensor_slices(
    x_train.reshape(60000, 784).astype("float32") / 255
)
dataset = dataset.map(lambda x: (x, x))  # Use x_train as both inputs & targets
dataset = dataset.shuffle(buffer_size=1024).batch(32)

# Configure the model for training.
vae.compile(optimizer, loss=loss_fn)

# Actually training the model.
vae.fit(dataset, epochs=1)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1875/1875 [==============================] - 19s 10ms/step - loss: 0.0714


time: 20.3 s (started: 2021-08-26 20:43:00 +08:00)


Functional API 和 fit 的使用将我们的示例从 65 行减少到 25 行（包括模型定义和训练）。 Keras 的理念是为您提供诸如此类的提高生产力的功能，同时使您能够自己编写所有内容，从而获得对每一个小细节的绝对控制。 就像我们在前两段的低级训练循环中所做的那样。

# 端到端实验示例 2：超网络。
让我们来看看另一种研究实验：超网络。

这个想法是使用一个小的深度神经网络（超网络）来为一个更大的网络（主网络）生成权重。

让我们实现一个非常简单的超网络：我们将使用一个小的 2 层网络来生成一个更大的 3 层网络的权重。

In [47]:
import numpy as np

input_dim = 784
classes = 10

# 这是我们实际用于预测标签的主要网络。
main_network = keras.Sequential(
    [keras.layers.Dense(64, activation=tf.nn.relu), keras.layers.Dense(classes),]
)

# 它不需要创建自己的权重，所以让我们将它的层标记为已经构建。 这样，调用 `main_network` 不会创建新变量。
for layer in main_network.layers:
    layer.built = True

# 这是要生成的权重系数的数量。 主网络中的每一层都需要 output_dim * input_dim + output_dim 系数。
num_weights_to_generate = (classes * 64 + classes) + (64 * input_dim + 64)

# 这是生成上面`main_network` 权重的超网络。
hypernetwork = keras.Sequential(
    [
        keras.layers.Dense(16, activation=tf.nn.relu),
        keras.layers.Dense(num_weights_to_generate, activation=tf.nn.sigmoid),
    ]
)

time: 9.2 ms (started: 2021-08-26 20:47:07 +08:00)


这是我们的训练循环。 对于每批数据：
* 我们使用超网络生成一组权重系数，weights_pred
* 我们将这些系数重塑为 main_network 的内核和偏置张量
* 我们运行 main_network 的前向传递来计算实际的 MNIST 预测
* 我们通过超网络的权重运行反向传播以最小化最终分类损失

In [48]:
# Loss and optimizer.
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)

# Prepare a dataset.
(x_train, y_train), _ = tf.keras.datasets.mnist.load_data()
dataset = tf.data.Dataset.from_tensor_slices(
    (x_train.reshape(60000, 784).astype("float32") / 255, y_train)
)

# We'll use a batch size of 1 for this experiment.
dataset = dataset.shuffle(buffer_size=1024).batch(1)


@tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        # Predict weights for the outer model.
        weights_pred = hypernetwork(x)

        # Reshape them to the expected shapes for w and b for the outer model.
        # Layer 0 kernel.
        start_index = 0
        w0_shape = (input_dim, 64)
        w0_coeffs = weights_pred[:, start_index : start_index + np.prod(w0_shape)]
        w0 = tf.reshape(w0_coeffs, w0_shape)
        start_index += np.prod(w0_shape)
        # Layer 0 bias.
        b0_shape = (64,)
        b0_coeffs = weights_pred[:, start_index : start_index + np.prod(b0_shape)]
        b0 = tf.reshape(b0_coeffs, b0_shape)
        start_index += np.prod(b0_shape)
        # Layer 1 kernel.
        w1_shape = (64, classes)
        w1_coeffs = weights_pred[:, start_index : start_index + np.prod(w1_shape)]
        w1 = tf.reshape(w1_coeffs, w1_shape)
        start_index += np.prod(w1_shape)
        # Layer 1 bias.
        b1_shape = (classes,)
        b1_coeffs = weights_pred[:, start_index : start_index + np.prod(b1_shape)]
        b1 = tf.reshape(b1_coeffs, b1_shape)
        start_index += np.prod(b1_shape)

        # Set the weight predictions as the weight variables on the outer model.
        main_network.layers[0].kernel = w0
        main_network.layers[0].bias = b0
        main_network.layers[1].kernel = w1
        main_network.layers[1].bias = b1

        # Inference on the outer model.
        preds = main_network(x)
        loss = loss_fn(y, preds)

    # Train only inner model.
#     grads = tape.gradient(loss, hypernetwork.trainable_weights)
#     optimizer.apply_gradients(zip(grads, hypernetwork.trainable_weights))
    optimizer.minimize(loss, hypernetwork.trainable_weights, tape = tape)
    return loss

losses = []  # Keep track of the losses over time.
for step, (x, y) in enumerate(dataset):
    loss = train_step(x, y)

    # Logging.
    losses.append(float(loss))
    if step % 100 == 0:
        print("Step:", step, "Loss:", sum(losses) / len(losses))

    # Stop after 1000 steps.
    # Training the model to convergence is left
    # as an exercise to the reader.
    if step >= 1000:
        break

Step: 0 Loss: 3.984729766845703
Step: 100 Loss: 2.755797770685784
Step: 200 Loss: 2.293525581169336
Step: 300 Loss: 2.1120521302282365
Step: 400 Loss: 1.9611971037511127
Step: 500 Loss: 1.8432813464796836
Step: 600 Loss: 1.71873571755215
Step: 700 Loss: 1.6866467812311818
Step: 800 Loss: 1.6358893030762025
Step: 900 Loss: 1.5616319959051859
Step: 1000 Loss: 1.4955019258477595
time: 6.2 s (started: 2021-08-26 20:49:34 +08:00)


使用 Keras 实现任意研究想法既简单又高效。 想象一下每天尝试 25 个想法（平均每个实验 20 分钟）！

Keras 旨在尽快将想法转化为结果，因为我们相信这是开展出色研究的关键。

我们希望您喜欢这个快速介绍。 让我们知道您使用 Keras 构建了什么！

In [49]:
tf.__version__

'2.5.0'

time: 3.57 ms (started: 2021-08-26 20:50:34 +08:00)
